In [1]:
import xarray as xr
import numpy as np

In [2]:
original = xr.open_dataset('/capstor/store/cscs/ERA5/weatherbench2_original', engine='zarr')
wind_original = original['10m_u_component_of_wind']
data = wind_original[:100, :, :].data

In [3]:
from lib import LIBWAVERANGE, NLAYMAX
import ctypes

In [9]:
ntot_enc_max = NLAYMAX * max(1024, np.prod(data.shape))

tolabs = ctypes.c_float()
midval = ctypes.c_float()
halfspanval = ctypes.c_float()
wlev = ctypes.c_uint8()
nlay = ctypes.c_uint8()
ntot_enc = ctypes.c_ulong()
cutoffval = ctypes.c_float(10000)
deps_vec = np.zeros(NLAYMAX, dtype=np.float32)
minval_vec = np.zeros(NLAYMAX, dtype=np.float32)
len_enc_vec = np.zeros(NLAYMAX, dtype=np.uint64)
data_enc = np.zeros(ntot_enc_max, dtype=np.uint8)

output = np.zeros_like(data, dtype=np.float32)
original = data.copy()

LIBWAVERANGE.encoding_wrap_float(data.shape[-1], data.shape[-2], data.shape[-3], data, 1, 1, 1, 1, ctypes.pointer(cutoffval), ctypes.byref(tolabs),
        ctypes.byref(midval), ctypes.byref(halfspanval), ctypes.byref(wlev), ctypes.byref(nlay), ctypes.byref(ntot_enc), deps_vec, minval_vec, len_enc_vec, data_enc)

data_enc = data_enc[:ntot_enc.value]

LIBWAVERANGE.decoding_wrap_float(data.shape[-1], data.shape[-2], data.shape[-3], output, ctypes.byref(tolabs),
        ctypes.byref(midval), ctypes.byref(halfspanval), ctypes.byref(wlev), ctypes.byref(nlay), ctypes.byref(ntot_enc), deps_vec, minval_vec, len_enc_vec, data_enc)

print(f'compressed to {ntot_enc.value} (from {original.nbytes}, ratio of {original.nbytes / ntot_enc.value})')
print(np.max(np.abs(output - original)))

compressed to 886493 (from 415296000, ratio of 468.47070422439884)
0.00023744203
